<a href="https://colab.research.google.com/github/danyesss/NLP/blob/main/corpus_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Russian language dataset

In [ ]:
!pip install langdetect tqdm
!pip install "tqdm==4.43.0"
!sudo apt-get install -y xsltproc
from fastai.basics import *
from tqdm import *
from tqdm.contrib.concurrent import process_map, thread_map
from multiprocessing import Pool
import regex as re
import time
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
 
librusec = '/home/u/nas/librusec/lib.rus.ec'
tmpzips = './tmp/zip'
tmptxt = './tmp/txt'
tmpfb2clean = './tmp/fb2clean'
tmpfb2unzip = './tmp/fb2unzip'
data = Path('../data/full')
 
!curl -LJO https://raw.githubusercontent.com/mgrankin/ru_transformers/master/corpus/FB2_2_txt.xsl

!mkdir ../data
!mkdir ../data/full
!mkdir ../data/classic
!mkdir tmp
!mkdir tmp/fb2unzip
!mkdir tmp/fb2clean
!mkdir tmp/txt
!mkdir tmp/zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xsltproc is already the newest version (1.1.29-5ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5003  100  5003    0     0  48105      0 --:--:-- --:--:-- --:--:-- 48105
mkdir: cannot create directory ‘../data’: File exists
mkdir: cannot create directory ‘../data/full’: File exists
mkdir: cannot create directory ‘../data/classic’: File exists
mkdir: cannot create directory

### Unpack ZIPs

Before running this we need to upload our fb2 zip files to /tmp/zip and then run this script

In [ ]:
# clean the output directory 
!rm -rfv {tmpfb2unzip + '/*'}

In [ ]:
zips = get_files(tmpzips, '.zip')
print(f'{len(zips)} zip file(s) found')


1 zip file(s) found


In [ ]:
def unpack(fn):
    !unzip -qq -joL -qo -O cp396 {fn} -d {tmpfb2unzip} >>/dev/null

# Unpack zips in parallel
thread_map(unpack, zips, max_workers=64)
print(f'Unzipped all - DONE')

# Sanitize file and folder names - remove spaces
!find $tmp -depth -name "* *" -execdir rename 's/ /_/g' "{}" \;


Unzipped all - DONE


### Convert fb2 to txt

In [ ]:
# clean the output directory 
!rm -rfv {tmpfb2clean + '/*'}

removed './tmp/fb2clean/Осинский_-_Виктор_Пелевин_и_эффект_Пустоты.fb2'
removed './tmp/fb2clean/Пелевин_1_Орден_желтого_флага.fb2'
removed './tmp/fb2clean/Пелевин_2_Железная_бездна.fb2'
removed './tmp/fb2clean/Пелевин_-_46_интервью_с_Пелевиным.fb2'
removed './tmp/fb2clean/Пелевин_-_Empire_V.fb2'
removed './tmp/fb2clean/Пелевин_-_Generation_«П».fb2'
removed './tmp/fb2clean/Пелевин_-_iPhuck_10.fb2'
removed './tmp/fb2clean/Пелевин_-_Relics_Раннее_и_неизданное_Сборник.fb2'
removed './tmp/fb2clean/Пелевин_-_SNUFF.fb2'
removed './tmp/fb2clean/Пелевин_-_t.fb2'
removed './tmp/fb2clean/Пелевин_-_Timeout_или_Вечерняя_Москва.fb2'
removed './tmp/fb2clean/Пелевин_-_Ultima_Тулеев_или_Дао_выборов.fb2'
removed './tmp/fb2clean/Пелевин_-_Who_by_fire.fb2'
removed './tmp/fb2clean/Пелевин_-_Акико.fb2'
removed './tmp/fb2clean/Пелевин_-_Ампир_«В».fb2'
removed './tmp/fb2clean/Пелевин_-_Ананасная_вода_для_прекрасной_дамы.fb2'
removed './tmp/fb2clean/Пелевин_-_Ассасин.fb2'
removed './tmp/fb2clean/Пелевин_-_Бубе

In [ ]:
# clean the output directory 
!rm -rfv {tmptxt + '/*'}

removed './tmp/txt/Осинский_-_Виктор_Пелевин_и_эффект_Пустоты.txt'
removed './tmp/txt/Пелевин_1_Орден_желтого_флага.txt'
removed './tmp/txt/Пелевин_2_Железная_бездна.txt'
removed './tmp/txt/Пелевин_-_46_интервью_с_Пелевиным.txt'
removed './tmp/txt/Пелевин_-_Empire_V.txt'
removed './tmp/txt/Пелевин_-_Generation_«П».txt'
removed './tmp/txt/Пелевин_-_iPhuck_10.txt'
removed './tmp/txt/Пелевин_-_Relics_Раннее_и_неизданное_Сборник.txt'
removed './tmp/txt/Пелевин_-_SNUFF.txt'
removed './tmp/txt/Пелевин_-_Timeout_или_Вечерняя_Москва.txt'
removed './tmp/txt/Пелевин_-_t.txt'
removed './tmp/txt/Пелевин_-_Ultima_Тулеев_или_Дао_выборов.txt'
removed './tmp/txt/Пелевин_-_Who_by_fire.txt'
removed './tmp/txt/Пелевин_-_Акико.txt'
removed './tmp/txt/Пелевин_-_Ампир_«В».txt'
removed './tmp/txt/Пелевин_-_Ананасная_вода_для_прекрасной_дамы.txt'
removed './tmp/txt/Пелевин_-_Ассасин.txt'
removed './tmp/txt/Пелевин_-_Бубен_верхнего_мира.txt'
removed './tmp/txt/Пелевин_-_Бубен_нижнего_мира.txt'
removed './tmp/t

In [ ]:
# Get fb2s
fbs = get_files(tmpfb2unzip, '.fb2', recurse=True)

# Sanitize filenames and move to the 'clean' dir
for fn in fbs:
    nn = (str(fn.name)
        .replace(' ','')
        .replace('_quot;','')
        .replace('!','')
        .replace(',','')
        .replace('(','')
        .replace(')','')
        .replace('\xa0','')
        .replace('.','')
        .replace('fb2', '.fb2')
         )
    shutil.move(fn, f'{tmpfb2clean}/{nn}')
print(f'{len(fbs)} fb2(s) sanitized')


4 fb2(s) sanitized


In [ ]:
# In order to convert you need to get the conversion xsl from here:
# https://github.com/mgrankin/ru_transformers/blob/master/corpus/FB2_2_txt.xsl
# and put it in the root folder
def convert_fb2(fn):
    #!xsltproc FB2_2_txt.xsl {fn} > {str(fn).replace(' ', '').replace('.fb2','.txt').replace('/fb2','/txt')} 2>>/dev/null
    !xsltproc FB2_2_txt.xsl {fn} > {tmptxt + '/' + fn.name.replace('fb2', 'txt')} 2>>/dev/null
    return {fn}

# Get fb2s from the clean snitized dir
fbs = get_files(tmpfb2clean, '.fb2')

# convert all to .txt
thread_map(convert_fb2, fbs, max_workers=64)
print('FB2(s) conversion done')


FB2(s) conversion done


### Filter and concat txt files

In [ ]:
txts = get_files('./tmp/txt', '.txt')
print(f'Found {len(txts)} txt(s)')

# this will take time, bcs langdetect fails on multithreading
print('Running langdetect . . . ')

for fn in progress_bar(txts):

    with open(f'./{fn}', 'r') as f:
        lines = f.read()
        print(f)
        print(fn)
        print(len(lines))
        try:
            if len(lines) > 1e+4 and detect(lines) == 'ru':
                with open(f'{data}/{fn.name}', 'w') as c:
                    c.write(lines)
        except LangDetectException as e:
            pass

# Add space before each word. It's not really nesessary. 
# It just makes encoding a bit more meaningful to the model and the text smaller(after encoding).
print('Running text sanitization . . . ')
def process_fn(fn):
    match = re.compile(r'(?=[^ ])([\W])([\w])')
    match2 = re.compile('(.|\s)\\1\\1+')
    with open(fn, 'r') as f:
        lines = f.read()
    if lines and lines[0] != ' ': lines = ' ' + lines
    lines = match.sub(r'\g<1> \g<2>', lines)
    lines = match2.sub(r'\1'*3, lines)
    with open(fn, 'w') as c:
        c.write(lines)
        
thread_map(process_fn, txts)

In [ ]:
txts = get_files('./tmp/txt', '.txt')
length = len(txts)
filter = np.ones(length, dtype=bool)
filter[::4] = False

In [ ]:
# txts = txts[filter]
txts = txts[::4]
print(f'Found {len(txts)} txt(s)')

# this will take time, bcs langdetect fails on multithreading
print('Running langdetect . . . ')
i=0
for fn in progress_bar(txts):
    if i%2 == 0:
      with open(f'./{fn}', 'r') as f:
          lines = f.read()
          print(f)
          print(fn)
          print(len(lines))
          try:
              if len(lines) > 1e+4 and detect(lines) == 'ru':
                  with open(f'{data}/{fn.name}', 'w') as c:
                      c.write(lines)
          except LangDetectException as e:
              pass
    i += 1

# Add space before each word. It's not really nesessary. 
# It just makes encoding a bit more meaningful to the model and the text smaller(after encoding).
print('Running text sanitization . . . ')
def process_fn(fn):
    match = re.compile(r'(?=[^ ])([\W])([\w])')
    match2 = re.compile('(.|\s)\\1\\1+')
    with open(fn, 'r') as f:
        lines = f.read()
    if lines and lines[0] != ' ': lines = ' ' + lines
    lines = match.sub(r'\g<1> \g<2>', lines)
    lines = match2.sub(r'\1'*3, lines)
    with open(fn, 'w') as c:
        c.write(lines)
        
thread_map(process_fn, txts)

Found 1 txt(s)
Running langdetect . . . 


<_io.TextIOWrapper name='./tmp/txt/Brodskiy_Sochineniya-Iosifa-Brodskogo-Pushkinskiy-Fond-_1_Sochineniya-Iosifa-Brodskogo-Tom-IEKQ90w656307.txt' mode='r' encoding='UTF-8'>
tmp/txt/Brodskiy_Sochineniya-Iosifa-Brodskogo-Pushkinskiy-Fond-_1_Sochineniya-Iosifa-Brodskogo-Tom-IEKQ90w656307.txt
315204
Running text sanitization . . . 


[None]

In [ ]:
# txts = get_files(data, '.txt'); 
# print(f'Amount of txt(s): {len(txts)}')

Amount of txt(s): 87


In [ ]:
fsorted = get_files('../data/classic', '.txt') + sorted(txts, key=lambda fn: os.path.getsize(fn))

print('Building text corpus')
sz=0
double_rate = 8
with open('./tmp/brodsky_valid.txt', 'w') as c:
    for iterator in range(double_rate):
        for fn in fsorted:
            with open(fn, 'r') as f:
                # sz += c.write(f.read().replace('\n', f' {NEW_LINE} ') + '\n')
                sz += c.write(f.read())
                if sz > 5e+9:
                    break
print('Done ./tmp/brodsky_valid.txt')

Building text corpus
Done ./tmp/brodsky_valid.txt
